## Integrating tools with Agents

In [3]:
from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from PyPDF2 import PdfReader
import gradio as gr

In [4]:
load_dotenv(override=True)

openai = OpenAI()

In [7]:
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [8]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data = payload)

In [9]:
push("hey there!")

Push: hey there!


Building tools in the form of functions that the LLM can call and use

In [12]:
def record_user_details(email, name="Name not provided", notes = "not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

In [16]:
def record_unknown_question(question):
    push(f"Recording {question} asked that I could not answer")
    return {"recorded": "ok"}

The following json calls the function tool we have described earlier and describes the capability of the tool/function 

This is the information that goes to OpenAI to work on

In [60]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record a user's interest in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "the email address of this user"
            },
            "name": {
                "type": "string",
                "description": "the user's name, if they provided it"
            },
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }            
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

In [61]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False 
    }
}

In [62]:
tools = [{"type": "function", "function": record_user_details_json },
        {"type": "function", "function": record_unknown_question_json }]

In [63]:
tools

[{'type': 'function',
  'function': {'name': 'record_user_details',
   'description': "Use this tool to record a user's interest in being in touch and provided an email address",
   'parameters': {'type': 'object',
    'properties': {'email': {'type': 'string',
      'description': 'the email address of this user'},
     'name': {'type': 'string',
      'description': "the user's name, if they provided it"},
     'notes': {'type': 'string',
      'description': "Any additional information about the conversation that's worth recording to give context"}},
    'required': ['email'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_unknown_question',
   'description': "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': "The question that couldn't be answered"}},
    'required': ['question'],
 

The following function will run when the LLM decides that it wants to run one of these tools

In [64]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush = True)

        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)

        results.append({"role": "tool", "content": json.dumps(result), "tool_call_id": tool_call.id})
    return results


In [25]:
globals()["record_unknown_question"]("this is a really hard question") #globals 

Push: Recording this is a really hard question asked that I could not answer


{'recorded': 'ok'}

Changing the fucnction to use "globals"

In [69]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush = True)
        tool = globals()[tool_name]
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool", "content": json.dumps(result), "tool_call_id": tool_call.id})
    return results

In [28]:
loc = r"C:\Users\KAUSTHAB DUTTA\Documents\practce_agentic_ai_project\projects\data_files\me.pdf"

In [55]:
reader = PdfReader(loc)

linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin = linkedin + text 



In [32]:
print(linkedin)

   
Contact
+14375564666  (Mobile)
kausthab.dutta1987@gmail.co
m
www.linkedin.com/in/kduttap
(LinkedIn)
Top Skills
Natural Language Processing (NLP)
Predictive Analytics
Datasets
Languages
Assamese  (Native or Bilingual)
Hindi  (Native or Bilingual)
Gujarati  (Limited Working)
English  (Full Professional)
Certifications
Certified NLP Coach
AMCAT Certified Proficiency in
English
SQL Intermediate
Analyzing and Visualizing Data with
Microsoft Power BI
Certificate of Completion - Assistant
Project Manager
Honors-Awards
Rising Star Award 2015Kausthab Dutta Phukan
Data Scientist Helping Others Transform Their Careers | Sales to
Data Science | Analytics Mentor | Product Development | Social
Impact through Analytics Projects
Hamilton, Ontario, Canada
Summary
As a seasoned professional with over 10 years of experience, I
specialize in data science, artificial intelligence (AI), and product
development. My expertise lies in applying deep learning and
computer vision techniques to solve real-worl

In [56]:
with open(r"C:\Users\KAUSTHAB DUTTA\Documents\practce_agentic_ai_project\projects\data_files\profile.txt","r") as f:
    summary = f.read()

name = "Kausthab"

In [40]:
summary

'Kausthab Dutta Phukan\nHamilton, Ontario, Canada\n+1 (437)-556-4666 | kausthab.dutta@outlook.com\nLinkedIn Profile | GitHub Profile\nPermanent Resident | Open to Hybrid Roles | Available Immediately________________________________________\nPROFESSIONAL SUMMARY\nData Scientist with 3+ years of experience in machine learning, advanced statistical methods, and data-driven decision-making. Proven expertise in developing predictive models, automating data pipelines, and creating actionable insights for stakeholders. Skilled in Python, TensorFlow, SQL, and Power BI, with a strong ability to collaborate across departments and translate complex data into strategic recommendations. Passionate about solving business problems through innovative data solutions and staying current with emerging trends in AI and machine learning.________________________________________\nTECHNICAL SKILLS\nâ€¢\tMachine Learning: Linear/Logistic Regression, Decision Trees, Random Forest, CNN, NLP, TensorFlow, scikit-l

In [57]:
system_prompt = f"""You are acting as {name}. You are answering questions on behalf of {name} on {name}'s career website. People will visit their website and \
interact with you and you will answer all questions truthfully and in an egaging manner. Be courteous and empathetic. If you do not know the answer to any \
question, use the record_unknown_question tool to record question even if that question is not related to {name}'s career. Also, if a user is engaging in a \
discussion, try to steer them towards getting in touch vis email; ask for their email and record their response using your record_user_details tool. In order \
to better equip you, you will be given context of {name}'s LinkedIn and a snapshot of their resume/profile, so that you can engage with potential recruiters \
or clinets in a professional and engaging manner"""

system_prompt += f"""\n\n Summary:\n{summary}\n\n## LinkedIn Profile: \n{linkedin}\n\n"""
system_prompt += f"with this context, please chat with the user, always staying in character as {name}."

In [37]:
system_prompt

"You are acting as Kausthab. You are answering questions on behalf of Kausthab on Kausthab's career website. People will visit their website and interact with you and you will answer all questions truthfully and in an egaging manner. Be courteous and empathetic. If you do not know the answer to any question, use the record_unknown_question tool to record question even if that question is not related to Kausthab's career. Also, if a user is engaging in a discussion, try to steer them towards getting in touch vis email; ask for their email and record their response using your record_user_details tool. In order to better equip you, you will be given context of Kausthab's LinkedIn and a snapshot of their resume/profile, so that you can engage with potential recruiters or clinets in a professional and engaging manner\n\n Summary:\nKausthab Dutta Phukan\nHamilton, Ontario, Canada\n+1 (437)-556-4666 | kausthab.dutta@outlook.com\nLinkedIn Profile | GitHub Profile\nPermanent Resident | Open to 

In [67]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False 
    while not done:
        response = openai.chat.completions.create(model = "gpt-4o-mini", messages=messages, tools=tools)

        finish_reason = response.choices[0].finish_reason

        if finish_reason == "tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls #take out the tool call
            results = handle_tool_calls(tool_calls) #function is called
            messages.append(message)
            messages.extend(results)
        else:
            done = True 
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(chat, type = "messages").launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


Tool called: record_unknown_question
Push: Recording What is your favorite color? asked that I could not answer
Tool called: record_user_details
Push: Recording interest from Name not provided with email kevin@test.com and notes not provided


: 